In [1]:
# 2 weeks together
import pandas as pd
import numpy as np
import datetime
import os
# Add the new folders of Jian later

In [2]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
            
last_saturday=datetime.datetime.now().date()-datetime.timedelta(days=datetime.datetime.now().date().weekday()+2) #+7

write_folder="/home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_"+str(last_saturday)+"/"
try:
    os.stat(write_folder)
except:
    os.mkdir(write_folder)

week_beginning=last_saturday-datetime.timedelta(days=14) 

recent_2_weeks=[last_saturday-datetime.timedelta(days=x*7) for x in range(2)]
recent_2_weeks

[datetime.date(2020, 1, 25), datetime.date(2020, 1, 18)]

In [3]:
# Checking for the recent weeks that haven't moved to the folder
possible_recent_folders=["/home/jian/BigLots/MediaStorm_"+str(x)+"/" for x in recent_2_weeks]
recent_file_list=[]
for dirc in possible_recent_folders:
    list_recent=[x for x in list(recursive_file_gen(dirc)) if ("Master" in x) & (".txt" in x) ]
    recent_file_list=recent_file_list+list_recent
recent_file_df=pd.DataFrame({"path":recent_file_list,"date":[datetime.datetime.strptime(x.split("MasterWeekly")[1][:8],"%Y%m%d").date()-datetime.timedelta(days=3) for x in recent_file_list]},index=[x for x in range(len(recent_file_list))])


list_1_after_201806=[x for x in list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/")) if ("Master" in x) & (".txt" in x) ]
list_2_after_201806=[x for x in list(recursive_file_gen("/home/jian/BigLots/2020_by_weeks/")) if ("Master" in x) & (".txt" in x) ]
list_1_after_201806=list_1_after_201806+list_2_after_201806
folder_date=[datetime.datetime.strptime(x.split("/")[len(x.split("/"))-2].split("_")[1],"%Y-%m-%d").date() for x in list_1_after_201806]
df_1_after_201806=pd.DataFrame({"date":folder_date,"path":list_1_after_201806},index=[x for x in range(len(list_1_after_201806))])
df_1_after_201806['date'].apply(lambda x: x.weekday()).unique()
df_1_after_201806=df_1_after_201806.sort_values("date").reset_index()
df_1_after_201806=df_1_after_201806[df_1_after_201806['date']>week_beginning]
del df_1_after_201806['index']

new_2_biweekly_files=df_1_after_201806.append(recent_file_df)

print(len(new_2_biweekly_files))
new_2_biweekly_files=new_2_biweekly_files['path'].unique().tolist()

2


/home/jian/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [4]:
new_2_biweekly_files

['/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-01-18/MediaStormMasterWeekly20200121-112014-195.txt',
 '/home/jian/BigLots/MediaStorm_2020-01-25/MediaStormMasterWeekly20200128-112021-627.txt']

In [5]:
if len(new_2_biweekly_files)==2: # Changed to weekly
    df_all_new_master = pd.DataFrame()
    for file in new_2_biweekly_files:
        df = pd.read_csv(file,nrows = None,sep= '|',usecols = ['customer_id_hashed','email_address_hash','customer_zip_code'],
                         dtype =str)
        df_all_new_master = df_all_new_master.append(df,ignore_index = True)
        print(df_all_new_master.shape)
    print(len(df_all_new_master.index))
    df_all_new_master = df_all_new_master.drop_duplicates('email_address_hash')
    print(len(df_all_new_master.index))


(102524, 3)
(204342, 3)
204342
204340


In [6]:
# Remove previous week ids

previous_id_files_lists=list(recursive_file_gen("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/"))

# previous_id_files_lists=[x for x in previous_id_files_lists if "Copy of 48 Stores_727.csv" not in x]
previous_id_files_lists=[x for x in previous_id_files_lists if ".csv" in x]
print(len(previous_id_files_lists))

178


In [7]:
previous_id_files_lists

['/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/T_Active(0-12)_Score(H)_Density(H)_Legacy_HolidayShoppers_P_2019Q4_RFM.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/T_Active(0-12)_Score(H)_Density(H)_Legacy_HolidayShoppers_S_2019Q4_RFM.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/T_Active(0-12)_Score(H)_Density(H)_Legacy_Others_P_2019Q4_RFM.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/T_Active(0-12)_Score(H)_Density(H)_Legacy_Others_S_2019Q4_RFM.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/T_Active(0-12)_Score(H)_Density(H)_SOTF_HolidayShoppers_P_2019Q4_RFM.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/T_Active(0-12)_Score(H)_Density(H)_SOTF_HolidayShoppers_S_2019Q4_RFM.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/T_Active(0-12)

In [8]:
import glob
uploaded_new_seg=list(recursive_file_gen("/home/jian/celery/BiWeekly_New_SignUps/monthly_update/"))
uploaded_new_seg=[x for x in uploaded_new_seg if ".csv" in x]
print("len(uploaded_new_seg)"+str(len(uploaded_new_seg)))
previous_id_files_lists=previous_id_files_lists+uploaded_new_seg

len(uploaded_new_seg)69


In [9]:
# Exclude the quarterly regular segments


In [10]:
for file_uploaded in previous_id_files_lists:
    df = pd.read_csv(file_uploaded,usecols = ['email_address_hash'])
    previous_email_set=set(df['email_address_hash'].tolist())
    df_all_new_master = df_all_new_master[~df_all_new_master['email_address_hash'].isin(previous_email_set)]
    print(datetime.datetime.now(),len(df_all_new_master.index),file_uploaded)

2020-01-30 15:34:06.629816 204340 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/T_Active(0-12)_Score(H)_Density(H)_Legacy_HolidayShoppers_P_2019Q4_RFM.csv
2020-01-30 15:34:06.689517 204340 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/T_Active(0-12)_Score(H)_Density(H)_Legacy_HolidayShoppers_S_2019Q4_RFM.csv
2020-01-30 15:34:07.981898 204340 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/T_Active(0-12)_Score(H)_Density(H)_Legacy_Others_P_2019Q4_RFM.csv
2020-01-30 15:34:08.302571 204340 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/T_Active(0-12)_Score(H)_Density(H)_Legacy_Others_S_2019Q4_RFM.csv
2020-01-30 15:34:08.352926 204340 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/T_Active(0-12)_Score(H)_Density(H)_SOTF_HolidayShoppers_P_2019Q4_RFM.csv
2020-01-30 15:34:08.402320 204340 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q

2020-01-30 15:35:47.717307 204309 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/T_Active(0-12)_Score(L)_Density(M)_SOTF_Others_P_2019Q4_RFM.csv
2020-01-30 15:35:48.510900 204309 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/T_Active(0-12)_Score(L)_Density(M)_SOTF_Others_S_2019Q4_RFM.csv
2020-01-30 15:35:48.653591 204309 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/T_Active(0-12)_Score(M)_Density(H)_Legacy_HolidayShoppers_P_2019Q4_RFM.csv
2020-01-30 15:35:48.721299 204309 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/T_Active(0-12)_Score(M)_Density(H)_Legacy_HolidayShoppers_S_2019Q4_RFM.csv
2020-01-30 15:35:49.648696 204309 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/T_Active(0-12)_Score(M)_Density(H)_Legacy_Others_P_2019Q4_RFM.csv
2020-01-30 15:35:49.957278 204309 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_fi

2020-01-30 15:37:09.902120 204294 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/C_Active(0-12)_Score(H)_Density(L)_SOTF_HolidayShoppers_P_2019Q4_RFM.csv
2020-01-30 15:37:09.940162 204294 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/C_Active(0-12)_Score(H)_Density(L)_SOTF_HolidayShoppers_S_2019Q4_RFM.csv
2020-01-30 15:37:10.016529 204294 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/C_Active(0-12)_Score(H)_Density(L)_SOTF_Others_P_2019Q4_RFM.csv
2020-01-30 15:37:10.070313 204294 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/C_Active(0-12)_Score(H)_Density(L)_SOTF_Others_S_2019Q4_RFM.csv
2020-01-30 15:37:10.171953 204294 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/C_Active(0-12)_Score(H)_Density(M)_Legacy_HolidayShoppers_P_2019Q4_RFM.csv
2020-01-30 15:37:10.278565 204294 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/uplo

2020-01-30 15:37:14.024020 204293 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/C_Active(0-12)_Score(M)_Density(L)_SOTF_HolidayShoppers_P_2019Q4_RFM.csv
2020-01-30 15:37:14.070001 204293 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/C_Active(0-12)_Score(M)_Density(L)_SOTF_HolidayShoppers_S_2019Q4_RFM.csv
2020-01-30 15:37:14.215903 204293 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/C_Active(0-12)_Score(M)_Density(L)_SOTF_Others_P_2019Q4_RFM.csv
2020-01-30 15:37:14.278155 204293 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/C_Active(0-12)_Score(M)_Density(L)_SOTF_Others_S_2019Q4_RFM.csv
2020-01-30 15:37:14.360673 204293 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/C_Active(0-12)_Score(M)_Density(M)_Legacy_HolidayShoppers_P_2019Q4_RFM.csv
2020-01-30 15:37:14.420618 204293 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/uplo

2020-01-30 15:38:44.161429 204279 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2019-05-04/NewReward_2019-05-04_P.csv
2020-01-30 15:38:44.302682 204279 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2019-05-04/NewReward_2019-05-04_S.csv
2020-01-30 15:38:44.371271 204279 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2019-05-04/NewReward_2019-05-04_T.csv
2020-01-30 15:38:44.929107 204279 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2019-05-18/NewReward_2019-05-18_P.csv
2020-01-30 15:38:45.110467 204279 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2019-05-18/NewReward_2019-05-18_S.csv
2020-01-30 15:38:45.210857 204279 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2019-05-18/NewReward_2019-05-18_T.csv
2020-01-30 15:38:45.623925 204279 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2019-06-01/NewReward_2019-06-01_P.csv
2020-01-30 15:38:45.770339 204279 /home/jian/celery/BiWeekly_New_Sign

In [11]:
df_all_new_master = df_all_new_master.drop_duplicates('email_address_hash')
df_all_new_master = df_all_new_master.drop_duplicates('customer_id_hashed')
print(len(df_all_new_master.index))

204274


In [12]:
zipmap = pd.ExcelFile('/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx')
zipmap=zipmap.parse("unique_zips_full_footprint",dtype=str)
zipmap['zip_type']=zipmap['zip_type'].replace("zips_10","trans_S")

zipmap['zipcodegroup'] = zipmap['zip_type'].apply(lambda x: x.split("_")[1])
print(zipmap.shape)
print(zipmap['zip_cd'].nunique())

zipmap=zipmap.rename(columns={"zip_cd":"customer_zip_code"})

zipmap=zipmap[['customer_zip_code','zipcodegroup']]
zipmap.head(2)

(12980, 3)
12980


,customer_zip_code,zipcodegroup
0,75040,P
1,20743,P


In [13]:
df_all_new_master = pd.merge(df_all_new_master,zipmap,on ='customer_zip_code',how = 'left' )
print(df_all_new_master['zipcodegroup'].unique())

df_all_new_master['zipcodegroup'].fillna('T',inplace = True)
print(df_all_new_master['zipcodegroup'].unique())

last_saturday_str="_"+str(last_saturday).split("-")[1]+str(last_saturday).split("-")[2]+str(last_saturday).split("-")[0][2:]+"_"

for revenue_flag in ['P','S','T']:
    df = df_all_new_master[df_all_new_master['zipcodegroup']==revenue_flag]
    df = df[['customer_id_hashed','email_address_hash','customer_zip_code']]
    df['segment_new_signups'] = 'NewReward' + last_saturday_str + revenue_flag
    print(revenue_flag,len(df.index))
    df.to_csv(write_folder+'NewReward_'+str(last_saturday)+'_'+revenue_flag+'.csv',index = False)
# To be uploaded to lr-big-lots /uploads/big_lots_onboarding

['P' 'S' nan]
['P' 'S' 'T']
P 90287
S 36662
T 77325


In [14]:
df_all_new_master.groupby(['zipcodegroup'])['email_address_hash'].count().to_frame().reset_index()

,zipcodegroup,email_address_hash
0,P,90287
1,S,36662
2,T,77325


In [18]:
df.head(2)

,customer_id_hashed,email_address_hash,customer_zip_code,segment_new_signups
5,3c2001c71e79569cc4d7a2dbfdf0daaecafd9ce5d38bc5...,6e720dc0b83cb248ded1338c004a4e3443ff7b4443b63e...,NaN,NewReward_012520_T
8,89c832384e7f735ecc6037378c530183bb4903a1a1ae0d...,52de4b40dfe80c007950abcc2b4d5a771e1337ec59a3d4...,64509,NewReward_012520_T


In [19]:
print(datetime.datetime.now())

2020-01-30 15:39:30.368476


In [20]:
print(datetime.datetime.now())

2020-01-30 15:39:31.737209


In [21]:
import paramiko
import glob

host = "files.liveramp.com"
port = 22
password = "Biglots2018!"
username = "lr-big-lots"

transport = paramiko.Transport((host, port))

transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)

# local_path defined above before saving the local txt
local_list=glob.glob(write_folder+"*.csv")
local_list=[x for x in local_list if 'NewReward_'+str(last_saturday)+'_' in x]

for local_path in local_list:
    remote_path="/uploads/big_lots_onboarding/"+os.path.basename(local_path)
    sftp.put(local_path,remote_path)
sftp.close()
transport.close()
print("Done: ",datetime.datetime.now())

Done:  2020-01-30 15:40:19.419225
